In [2]:
import folium

In [3]:
import json
import pandas as pd
import re
import geopandas as gpd

## Property location

In [108]:
# we want to draw a map to show all the locations of the extracted properties
m = folium.Map([-38.013995, 145.264296], zoom_start=9)

df = pd.read_csv('../data/curated/property.csv')

for index, row in df.iterrows():
    lat = float(re.findall(r'\[(-\d+.\d+)',str(row['coordinates']))[0])
    long = float(re.findall(r'(\d+.\d+)\]',str(row['coordinates']))[0])
    folium.Marker([lat, long]).add_to(m)

m.save('../plots/property.html')

## Property location with defined boundary of SA2

In [110]:
# we add boundary of each Statistical Area 2 into the property map
gdf = gpd.read_file('../data/curated/boundary.shp')
gdf = gpd.GeoDataFrame(gdf)
# create a JSON 
geoJSON = gdf[['SA2_CODE21', 'geometry']].to_json()

m.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
))
m.save('../plots/property_sa2.html')

In [111]:
map = folium.Map([-38.013995, 145.264296], zoom_start=9)

map.add_child(folium.Choropleth(
    geo_data=geoJSON,
    name='choropleth',
))
map.save('../plots/sa2.html')

## Property location with school location

In [4]:
import numpy as np
import math

In [113]:
df = pd.read_csv('../data/curated/school.csv')

for index, row in df.iterrows():
    folium.CircleMarker(location=(row['Y'], row['X']), radius=1, fill_color='red').add_to(m)

m.save('../plots/school_property.html')

## 2021 Population Density

In [5]:
import csv

In [104]:
# we draw a map to represent the density of population in each SA2
tmp_lst = []
with open('../data/curated/population.csv','r') as f:
    reader = csv.reader(f)
    for row in reader:
        tmp_lst.append(row)
df_popu = pd.DataFrame(tmp_lst[1:], columns=tmp_lst[0])

gdf = gpd.read_file('../data/curated/boundary.shp')
gdf = gpd.GeoDataFrame(gdf)
gdf.rename(columns={'SA2_CODE21':'SA2 code'}, inplace=True)

merge_df = pd.merge(df_popu,gdf,on='SA2 code')
merge_df = merge_df[['SA2 code', 'SA2 name', 'Population density 2021','geometry']]
new = merge_df['Population density 2021'].astype(float)
merge_df['density'] = new

# create a JSON 
gdf = gpd.GeoDataFrame(merge_df)
geoJSON = gdf[['SA2 code', 'geometry']].to_json()

m = folium.Map([-38.013995, 145.264296], zoom_start=9)

c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=merge_df, # data source
    columns=['SA2 code', 'density'], # the columns required
    key_on='properties.SA2 code', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Population density in 2021'
)

c.add_to(m)

m.save('../plots/population_density.html')


## Median Income 2018-2019

In [105]:
tmp_lst = []
with open('../data/curated/income.csv','r') as f:
    reader = csv.reader(f)
    for row in reader:
        tmp_lst.append(row)
df_income = pd.DataFrame(tmp_lst[1:], columns=tmp_lst[0])

gdf = gpd.read_file('../data/curated/boundary.shp')
gdf = gpd.GeoDataFrame(gdf)
gdf.rename(columns={'SA2_CODE21':'SA2'}, inplace=True)

merge_df = pd.merge(df_income,gdf,on='SA2',how='right')
merge_df = merge_df[['SA2', 'SA2_NAME21', '2018-19 MEDIAN EMPLOYMENT INCOME PER JOB','geometry']]


new = []
for i in range(len(merge_df)):
    if merge_df['2018-19 MEDIAN EMPLOYMENT INCOME PER JOB'][i] == 'np':
        new.append(0)
    else:
        new.append(float(merge_df['2018-19 MEDIAN EMPLOYMENT INCOME PER JOB'][i]))

merge_df['income'] = new

# create a JSON 
gdf = gpd.GeoDataFrame(merge_df)
geoJSON = gdf[['SA2', 'geometry']].to_json()

m = folium.Map([-38.013995, 145.264296], zoom_start=9)

c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=merge_df, # data source
    columns=['SA2', 'income'], # the columns required
    key_on='properties.SA2', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Median Income in 2018-2019'
)

c.add_to(m)

m.save('../plots/income.html')

## Mean Rental Price

In [17]:
tmp_lst = []
with open('../data/curated/rent_all.csv','r') as f:
    reader = csv.reader(f)
    for row in reader:
        tmp_lst.append(row)
df = pd.DataFrame(tmp_lst[1:], columns=tmp_lst[0])

new = df['SA2_code'].astype(float).astype(int).astype(str)
df['SA2_code'] = new

gdf = gpd.read_file('../data/curated/boundary.shp')
gdf = gpd.GeoDataFrame(gdf)
gdf.rename(columns={'SA2_CODE21':'SA2_code'}, inplace=True)


merge_df = pd.merge(df,gdf,on='SA2_code',how='right')
merge_df = merge_df[['SA2_code', 'SA2_NAME21', 'mean','geometry']]

new = []
for i in range(len(merge_df)):
    if merge_df['mean'][i] == 'np':
        new.append(0)
    else:
        new.append(float(merge_df['mean'][i]))

merge_df['mean'] = new

# create a JSON 
gdf = gpd.GeoDataFrame(merge_df)
geoJSON = gdf[['SA2_code', 'geometry']].to_json()

m = folium.Map([-38.013995, 145.264296], zoom_start=9)

c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=merge_df, # data source
    columns=['SA2_code', 'mean'], # the columns required
    key_on='properties.SA2_code', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Mean Rental Price for each SA2'
)

c.add_to(m)

m.save('../plots/mean_rent.html')

## Livable Suburb Rank

In [50]:
tmp_lst = []
with open('../data/curated/livable_suburb_all_2.csv','r') as f:
    reader = csv.reader(f)
    for row in reader:
        tmp_lst.append(row)
df = pd.DataFrame(tmp_lst[1:], columns=tmp_lst[0])


gdf = gpd.read_file('../data/curated/boundary.shp')
gdf = gpd.GeoDataFrame(gdf)
gdf.rename(columns={'SA2_NAME21':'SA2_name'}, inplace=True)


merge_df = pd.merge(df,gdf,on='SA2_name',how='right')
merge_df = merge_df[['SA2_CODE21', 'SA2_name', 'rank_avg','geometry']]

new = []
for i in range(len(merge_df)):
    if merge_df['rank_avg'][i] == 'np':
        new.append(0)
    else:
        new.append(float(merge_df['rank_avg'][i]))

merge_df['rank'] = new

# create a JSON 
gdf = gpd.GeoDataFrame(merge_df)
geoJSON = gdf[['SA2_name', 'geometry']].to_json()

m = folium.Map([-38.013995, 145.264296], zoom_start=9)

c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=merge_df, # data source
    columns=['SA2_name', 'rank'], # the columns required
    key_on='properties.SA2_name', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Rank of Distance to Infrastructure from Properties in each SA2'
)

c.add_to(m)



In [51]:
# we want to mark the top 20 most livable suburbs on the map

import numpy as np

tmp_lst = []
with open('../data/curated/livable_suburb_2_top_20.csv','r') as f:
    reader = csv.reader(f)
    for row in reader:
        tmp_lst.append(row)
df = pd.DataFrame(tmp_lst[1:], columns=tmp_lst[0])

df_merge = pd.merge(df,gdf,on='SA2_name',how='inner')

df_merge = df_merge[['SA2_name', 'rank','geometry']]

for i in range(len(df_merge)):
    loc = np.asarray(df_merge['geometry'][i].exterior)
    loc_list = [(i[1], i[0]) for i in loc]
    folium.vector_layers.Polygon(locations=loc_list, popup=df_merge['SA2_name'][i]).add_to(m)


m.save('../plots/livable_rank.html')

/var/folders/1l/kxzgd7qj0958pcn5xj1149ph0000gn/T/ipykernel_4251/2478040751.py:17: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  loc = np.asarray(df_merge['geometry'][i].exterior)


## Livable Suburb Distance

In [16]:
tmp_lst = []
with open('../data/curated/livable_suburb_all_1.csv','r') as f:
    reader = csv.reader(f)
    for row in reader:
        tmp_lst.append(row)
df = pd.DataFrame(tmp_lst[1:], columns=tmp_lst[0])


gdf = gpd.read_file('../data/curated/boundary.shp')
gdf = gpd.GeoDataFrame(gdf)
gdf.rename(columns={'SA2_NAME21':'SA2_name'}, inplace=True)


merge_df = pd.merge(df,gdf,on='SA2_name',how='right')
merge_df = merge_df[['SA2_CODE21', 'SA2_name', 'Distance_avg','geometry']]

new = []
for i in range(len(merge_df)):
    if merge_df['Distance_avg'][i] == 'np':
        new.append(0)
    else:
        new.append(float(merge_df['Distance_avg'][i]))

merge_df['distance'] = new

# create a JSON 
gdf = gpd.GeoDataFrame(merge_df)
geoJSON = gdf[['SA2_name', 'geometry']].to_json()

m = folium.Map([-38.013995, 145.264296], zoom_start=9)

c = folium.Choropleth(
    geo_data=geoJSON, # geoJSON 
    name='choropleth', # name of plot
    data=merge_df, # data source
    columns=['SA2_name', 'distance'], # the columns required
    key_on='properties.SA2_name', # this is from the geoJSON's properties
    fill_color='YlOrRd', # color scheme
    nan_fill_color='black',
    legend_name='Average Distance to Infrastructure from Properties in each SA2'
)

c.add_to(m)

m.save('../plots/livable_distance.html')